In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

In [31]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False



In [41]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# prediction_layer = tf.keras.layers.Dense(1)
prediction_layer = tf.keras.layers.Dense(2, activation='softmax')

In [43]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [44]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [47]:
TRAINING_DIR = "facemask-dataset/dest_folder/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(160, 160))

VALIDATION_DIR = "facemask-dataset/dest_folder/val"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(160, 160))


checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

Found 649 images belonging to 2 classes.
Found 142 images belonging to 2 classes.


In [48]:
history = model.fit_generator(train_generator,
                              epochs=15,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/15
64/65 [============================>.] - ETA: 2s - loss: 0.6924 - accuracy: 0.6495WARNING:tensorflow:From C:\Users\aamico\AppData\Local\Continuum\anaconda3\envs\documented\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model-001.model\assets
65/65 [==============================] - 195s 3s/step - loss: 0.6919 - accuracy: 0.6487 - val_loss: 0.6241 - val_accuracy: 0.8592
Epoch 2/15
65/65 [==============================] - 145s 2s/step - loss: 0.6566 - accuracy: 0.7658 - val_loss: 0.6068 - val_accuracy: 0.8662
Epoch 3/15
65/65 [==============================] - 152s 2s/step - loss: 0.6229 - accuracy: 0.8444 - val_loss: 0.5879 - val_accuracy: 0.9296
Epoch 4/15
65/65 [============

In [2]:
model = tf.keras.models.load_model('model-015.model')

## From tf to tf-lite

In [ ]:
export_dir = 'model-015.model'

# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
    f.write(tflite_model)